# Krylov Subspace Approximation XL-BOMD (KSA-XL-BOMD)
### see https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import KSA_XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [4]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/5_KSA_XL-BOMD'}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params,
                   seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)

md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   280.01   1.447747e-01 -1.350262e+00 -1.205487e+00 ||    377.92   1.954009e-01 1.466852e+00 1.662253e+00 || 
     2   241.13   1.246738e-01 -1.330652e+00 -1.205978e+00 ||    338.43   1.749812e-01 1.487071e+00 1.662052e+00 || 
     3   192.53   9.954737e-02 -1.305245e+00 -1.205698e+00 ||    328.68   1.699414e-01 1.492090e+00 1.662032e+00 || 
     4   140.56   7.267446e-02 -1.277607e+00 -1.204932e+00 ||    347.27   1.795506e-01 1.482432e+00 1.661983e+00 || 
     5    91.24   4.717297e-02 -1.251874e+00 -1.204701e+00 ||    388.91   2.010798e-01 1.460582e+00 1.661662e+00 || 
     6    50.89   2.631312e-02 -1.230833e+00 -1.204520e+00 ||    445.67   2.304291e-01 1.430972e+00 1.661401e+00 || 
     7    25.21   1.303693e-02 -1.217585e+00 -1.204548e+00 ||    507.97   2.626409e-01 1.398333e+00 1.660974e+00 || 
     8    17.43   9.010715e-03 -1.213532e+00 -1.204522e+00 ||    565.87  

## Damping (Langevin)

In [5]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/5_KSA_XL-BOMD_damp'}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100,
                   seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)

md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   285.79   1.477640e-01 -1.350202e+00 -1.202438e+00 ||    380.21   1.965849e-01 1.468534e+00 1.665119e+00 || 
     2   253.05   1.308349e-01 -1.329784e+00 -1.198949e+00 ||    364.13   1.882682e-01 1.489855e+00 1.678123e+00 || 
     3   213.17   1.102190e-01 -1.302888e+00 -1.192669e+00 ||    362.56   1.874566e-01 1.496260e+00 1.683717e+00 || 
     4   172.46   8.917040e-02 -1.271467e+00 -1.182296e+00 ||    394.61   2.040299e-01 1.487866e+00 1.691896e+00 || 
     5   124.15   6.419060e-02 -1.240592e+00 -1.176401e+00 ||    458.67   2.371496e-01 1.464524e+00 1.701674e+00 || 
     6    71.02   3.671826e-02 -1.213821e+00 -1.177102e+00 ||    515.63   2.666028e-01 1.436039e+00 1.702642e+00 || 
     7    29.70   1.535669e-02 -1.197851e+00 -1.182495e+00 ||    573.24   2.963897e-01 1.405435e+00 1.701825e+00 || 
     8    13.91   7.194288e-03 -1.192526e+00 -1.185332e+00 ||    611.22  